In [1]:
import sys
sys.path.append("../")

In [2]:
from datetime import date, datetime, timedelta
from sqlalchemy import Select, func, extract, or_, desc
import pandas as pd

In [3]:
import Logic.API_calls as ac
from Models.models import engine
from Models.models import od_dq_base, dq_agg_view, dq_agg_sum
import Logic.base_queries as bq

Loaded Env from File.
All Required Environment variables are loaded.
All Required Environment variables are loaded.


In [4]:
start_date = bq.get_date_range()[1]
prev_dt = start_date - timedelta(days=1)
print(start_date, prev_dt)

2024-06-25 2024-06-24


In [5]:
def query_top_cards(curr_dt: datetime.date, prev_dt: datetime.date) -> pd.DataFrame:
    top_cards = (
        Select(
            dq_agg_sum.c.ord_date.label("Order_Date"),
            func.sum(dq_agg_sum.c.total_orders).label("Total_Orders"),
            func.sum(dq_agg_sum.c.total_canceled_orders).label("Cancelled_Orders")
        ).where(or_(dq_agg_sum.c.ord_date == curr_dt, dq_agg_sum.c.ord_date == prev_dt))
        .group_by(dq_agg_sum.c.ord_date)
        .order_by(desc(dq_agg_sum.c.ord_date))
    )
    result = bq.run_stmt(top_cards)
    return pd.DataFrame(result)

In [6]:
query_top_cards(start_date, prev_dt)

,Order_Date,Total_Orders,Cancelled_Orders
0,2024-06-25,200731,3796
1,2024-06-24,209041,3582


In [10]:
import json
json_file = "../API/api_struct.json"

In [11]:
with open(json_file, "r") as f:
    data = f.read()
result = json.loads(data)

In [16]:
result["dq_report"]["cancel_highest_missing_pid_data"]

[{'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'},
 {'id': 'ondc-otipy.crofarm.com',
  'count': '0.81%',
  'increased': True,
  'variancePercentage': '1.2%',
  'varianceText': 'vs. Yesterday'}]